In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'brData10class14DomainReg.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Test set (1000, 28, 28) (1000,)


In [8]:
image_size = 28
#num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

print ()
print (train_labels[0:100])

Training set (200000, 28, 28, 1) (200000,)
Test set (1000, 28, 28, 1) (1000,)

[ 0.88235295  0.41176471  0.          0.29411766  0.64705884  0.05882353
  0.17647059  0.52941179  0.41176471  1.          0.17647059  0.88235295
  0.05882353  0.88235295  0.52941179  0.41176471  0.29411766  0.          0.7647059
  0.05882353  0.88235295  0.          0.52941179  0.64705884  0.29411766
  0.          0.52941179  0.05882353  1.          0.29411766  0.17647059
  0.17647059  0.88235295  0.41176471  0.41176471  0.64705884  0.41176471
  0.64705884  0.52941179  0.          0.64705884  0.29411766  0.41176471
  0.7647059   0.          0.          0.17647059  0.          0.52941179
  0.17647059  0.64705884  0.41176471  1.          0.7647059   0.41176471
  0.41176471  0.05882353  0.17647059  0.88235295  0.29411766  0.05882353
  0.88235295  0.64705884  0.17647059  0.41176471  0.7647059   0.29411766
  0.41176471  1.          0.          0.29411766  0.41176471  0.
  0.88235295  0.05882353  1.          1.  

In [9]:
def accuracy(predictions, values):
  return (100.0 * np.sum(predictions / values)
          / predictions.shape[0])

def meanSqrError(predictions, values):
  return np.mean(((predictions - values) ** 2))


In [49]:
batch_size = 1
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,))
  tf_test_dataset = tf.constant(test_dataset)
  tf_test_labels = tf.constant(test_labels)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, 1], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[1]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.square(logits - tf_train_labels)){:.9f}
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = logits
  test_prediction = model(tf_test_dataset)

SyntaxError: invalid syntax (<ipython-input-49-c97b3acca7f9>, line 44)

In [50]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size),]
    #print("batch_labels", batch_labels)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    #_, l, predictions = session.run(
    #  [optimizer, loss, train_prediction], feed_dict=feed_dict)
    tuning_cost = session.run(loss, feed_dict)
    #print ("Tuning cost=", "{:.9f}".format(tuning_cost))
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, tuning_cost))
  #print('Test accuracy: %.1f%%' % meanSqrError(test_prediction.eval(), test_prediction))
  #testing_cost = session.run(loss, feed_dict={tf_test_dataset: tf_test_dataset, test_labels:test_labels})    
  #print ("Testing data cost:" , testing_cost)

Initialized
Minibatch loss at step 0: 3066.517578
Minibatch loss at step 50: 3710.710205
Minibatch loss at step 100: 2893.981934
Minibatch loss at step 150: 3735.868652
Minibatch loss at step 200: 3234.356445
Minibatch loss at step 250: 4006.107910
Minibatch loss at step 300: 3563.303711
Minibatch loss at step 350: 3442.473633
Minibatch loss at step 400: 3467.869141
Minibatch loss at step 450: 3297.407471
Minibatch loss at step 500: 3330.419922
Minibatch loss at step 550: 3710.636719
Minibatch loss at step 600: 3208.737793
Minibatch loss at step 650: 3245.876465
Minibatch loss at step 700: 3662.324951
Minibatch loss at step 750: 3441.035645
Minibatch loss at step 800: 2941.825195
Minibatch loss at step 850: 3860.793213
Minibatch loss at step 900: 3535.318848
Minibatch loss at step 950: 3024.438965
Minibatch loss at step 1000: 3782.322754
Minibatch loss at step 1050: 3390.815918
Minibatch loss at step 1100: 3876.671387
Minibatch loss at step 1150: 3530.265869
Minibatch loss at step 1200

In [31]:
predictions.shape
batch_labels.shape

(150,)

In [16]:
train_labels[10:20,]

array([ 0.17647059,  0.88235295,  0.05882353,  0.88235295,  0.52941179,
        0.41176471,  0.29411766,  0.        ,  0.7647059 ,  0.05882353], dtype=float32)